In [1]:
import graphlab

# Load data

In [24]:
train_data = graphlab.SFrame.read_json("./train.json")

Finished parsing file /Users/xiaotianmu/Desktop/Kaggle/whats cooking/train.json

Parsing completed. Parsed 1 lines in 0.491291 secs.

In [25]:
test_data = graphlab.SFrame.read_json("./test.json")

Finished parsing file /Users/xiaotianmu/Desktop/Kaggle/whats cooking/test.json

Parsing completed. Parsed 1 lines in 0.121618 secs.

# 看一下数据是什么样

In [26]:
train_data

cuisine,id,ingredients
greek,10259,"[romaine lettuce, blackolives, grape tomatoes, ..."
southern_us,25693,"[plain flour, groundpepper, salt, tomatoes, ..."
filipino,20130,"[eggs, pepper, salt,mayonaise, cooking oil, ..."
indian,22213,"[water, vegetable oil,wheat, salt] ..."
indian,13162,"[black pepper, shallots,cornflour, cayenne ..."
jamaican,6602,"[plain flour, sugar,butter, eggs, fresh ..."
spanish,42779,"[olive oil, salt, mediumshrimp, pepper, garlic, ..."
italian,3735,"[sugar, pistachio nuts,white almond bark, fl ..."
mexican,16903,"[olive oil, purple onion,fresh pineapple, pork, ..."
italian,12734,"[chopped tomatoes, freshbasil, garlic, extra- ..."


In [27]:
test_data

id,ingredients
18009,"[baking powder, eggs,all-purpose flour, ..."
28583,"[sugar, egg yolks, cornstarch, cream of tartar, ..."
41580,"[sausage links, fennelbulb, fronds, olive oil, ..."
29752,"[meat cuts, file powder,smoked sausage, okra, ..."
35687,"[ground black pepper,salt, sausage casings, ..."
38527,"[baking powder, all-purpose flour, peach ..."
19666,"[grape juice, orange,white zinfandel] ..."
41217,"[ground ginger, whitepepper, green onions, ..."
28753,"[diced onions, tacoseasoning mix, all- ..."
22659,"[eggs, cherries, dates,dark muscovado sugar, ..."


# 使用TF-IDF对成分进行处理

In [17]:
train_data['tf_idf'] = graphlab.text_analytics.tf_idf(train_data['ingredients'])

In [18]:
test_data['tf_idf'] = graphlab.text_analytics.tf_idf(test_data['ingredients'])

# 处理之后的数据及显示

In [19]:
train_data

cuisine,id,ingredients,tf_idf
greek,10259,"[romaine lettuce, blackolives, grape tomatoes, ...","{'pepper':2.1930096079769075, ..."
southern_us,25693,"[plain flour, groundpepper, salt, tomatoes, ...","{'vegetable oil':2.205023806663163, 'g ..."
filipino,20130,"[eggs, pepper, salt,mayonaise, cooking oil, ...","{'garlic powder':3.3171823936245524, ..."
indian,22213,"[water, vegetable oil,wheat, salt] ...","{'vegetable oil':2.205023806663163, ..."
indian,13162,"[black pepper, shallots,cornflour, cayenne ...","{'butter':2.1048534756240707, ..."
jamaican,6602,"[plain flour, sugar,butter, eggs, fresh ...","{'butter':2.1048534756240707, ..."
spanish,42779,"[olive oil, salt, mediumshrimp, pepper, garlic, ...","{'chorizo sausage':5.95623972323981, 'olive ..."
italian,3735,"[sugar, pistachio nuts,white almond bark, fl ...","{'olive oil':1.6074034770415235, ..."
mexican,16903,"[olive oil, purple onion,fresh pineapple, pork, ...","{'olive oil':1.6074034770415235, ..."
italian,12734,"[chopped tomatoes, freshbasil, garlic, extra- ...","{'chopped tomatoes':4.8938752249640425, ..."


In [20]:
test_data

id,ingredients,tf_idf
18009,"[baking powder, eggs,all-purpose flour, ...","{'eggs':2.4316442575350115, ..."
28583,"[sugar, egg yolks, cornstarch, cream of tartar, ...","{'cream of tartar':5.420534999272286, 'corn ..."
41580,"[sausage links, fennelbulb, fronds, olive oil, ...","{'olive oil':1.6462078901449024, ..."
29752,"[meat cuts, file powder,smoked sausage, okra, ...","{'vegetable oil':2.173867157074427, ..."
35687,"[ground black pepper,salt, sausage casings, ...",{'parmigiano reggianocheese': ...
38527,"[baking powder, all-purpose flour, peach ...","{'corn starch':3.045629244698614, 'p ..."
19666,"[grape juice, orange,white zinfandel] ...","{'orange':4.773907834347233, 'g ..."
41217,"[ground ginger, whitepepper, green onions, ...","{'vegetable oil':2.173867157074427, 'w ..."
28753,"[diced onions, tacoseasoning mix, all- ...","{'vegetable oil':2.173867157074427, 'taco ..."
22659,"[eggs, cherries, dates,dark muscovado sugar, ...","{'butter':2.089955184824084, 'self ..."


In [23]:
graphlab.canvas.set_target('ipynb')
train_data.show()

# 对SVC, RFC,GBC使用cross_validation

In [29]:
folds = graphlab.cross_validation.KFold(train_data, 10)

In [65]:
params = dict([('target', 'cuisine'), ('max_depth', range(6,15))])
job = graphlab.model_parameter_search.create(folds,
                                        graphlab.random_forest_classifier.create,
                                        params)

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Feb-23-2017-11-14-4400000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Feb-23-2017-11-14-4400000' scheduled.


KeyboardInterrupt: 

In [64]:
print job.get_result()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
res.column_names()